In [1]:
!nvidia-smi

Wed Nov 27 22:42:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    Off | 00000000:01:00.0 Off |                  N/A |
| N/A   35C    P3              N/A /  40W |      7MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision.io import read_image
import glob
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import timm
import dlib
import cv2
import os
from IPython import embed

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [64]:
def convert_and_trim_bb(image, rect):
	startX = rect.left()
	startY = rect.top()
	endX = rect.right()
	endY = rect.bottom()
    
	startX = max(0, startX)
	startY = max(0, startY)
	endX = min(endX, image.shape[1])
	endY = min(endY, image.shape[0])
    
	w = endX - startX
	h = endY - startY
	return (startX, startY, w, h)

def get_frames(input_path,output_path,file_name):
    cap = cv2.VideoCapture(input_path)
    frame_id = 0
    frame_skip = 60
    frame_count = 0
    cropped = None
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_id > frame_skip - 1:
            frame_count += 1
            boxes = get_face_landmarks(frame)
            if len(boxes) > 0:
                try:
                    cropped = get_cropped_image(frame, boxes)
                except:
                    continue
            output_file =  file_name +'_'+ str(frame_count) +'.jpg'
            if cropped is not None:
                try:
                    cv2.imwrite(os.path.join(output_path, output_file), cropped)
                except:
                    continue
            frame_id = 0
            print(input_path + '/' + output_file)
            continue
        frame_id += 1
        
    cap.release()
    cv2.destroyAllWindows()

def get_face_landmarks(image):
    detector = dlib.get_frontal_face_detector()
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rects = detector(rgb)
    boxes = [convert_and_trim_bb(image, r) for r in rects]
    return boxes

def get_cropped_image(image, boxes):
    height,  width = image.shape[0], image.shape[1]
    scale_factor = 1.5
    for (x, y, w, h) in boxes:
        new_h = h * scale_factor
        new_w = w * scale_factor
        delta_h = (new_h - h) / 2
        delta_w = (new_w - w) / 2
        new_y = abs(y - delta_h)
        new_x = abs(x - delta_w)
        cropped = image[int(new_y):int(new_y+new_h), int(new_x):int(new_x+new_w)]
    return cropped

dataset_directory = "./Celeb-DF-v2/"

(images, directories, id) = ([], {}, 0)

for (subdirs, dirs, files) in os.walk(dataset_directory):
    print(subdirs)
    for subdir in dirs:
        directories[id] = subdir
        subjectpath = os.path.join(dataset_directory, subdir)
        output_path = subjectpath + '/' + subdir+ '_images'
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        for filename in os.listdir(subjectpath):
            if filename.endswith('.mp4'):
                input_path = subjectpath + '/' + filename
                file_name = filename.split('.')[0]
                get_frames(input_path,output_path,file_name)
        id += 1

./Celeb-DF-v2/
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_1.jpg
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_2.jpg
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_3.jpg
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_4.jpg
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_5.jpg
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_6.jpg
./Celeb-DF-v2/Celeb-real/id2_0000.mp4/id2_0000_7.jpg
./Celeb-DF-v2/Celeb-real/id6_0001.mp4/id6_0001_1.jpg
./Celeb-DF-v2/Celeb-real/id6_0001.mp4/id6_0001_2.jpg
./Celeb-DF-v2/Celeb-real/id6_0001.mp4/id6_0001_3.jpg
./Celeb-DF-v2/Celeb-real/id6_0001.mp4/id6_0001_4.jpg
./Celeb-DF-v2/Celeb-real/id6_0001.mp4/id6_0001_5.jpg
./Celeb-DF-v2/Celeb-real/id7_0000.mp4/id7_0000_1.jpg
./Celeb-DF-v2/Celeb-real/id7_0000.mp4/id7_0000_2.jpg
./Celeb-DF-v2/Celeb-real/id7_0000.mp4/id7_0000_3.jpg
./Celeb-DF-v2/Celeb-real/id7_0000.mp4/id7_0000_4.jpg
./Celeb-DF-v2/Celeb-real/id2_0002.mp4/id2_0002_1.jpg
./Celeb-DF-v2/Celeb-real/id2_0002.mp4/id2_0002_2.jpg
./Celeb-DF-v2/Celeb-real/id2_00

In [ ]:
class dataset_customizer(Dataset):
    def __init__(self, transform=None):
        self.imgs_path = '/home/utku/AI-Powered-Deepfake-Detection/Detected_face_images/Celeb-DF-v2_images/'
        self.transform = transform
        file_list = glob.glob(os.path.join(self.imgs_path, "*"))  # Get directories
        self.data = []
        self.class_map = {"Celeb-real_images": 0, "Celeb-synthesis_images": 1}

        for class_path in file_list:
            class_name = os.path.basename(class_path)  # Cross-platform folder name
            if class_name not in self.class_map:
                print(f"Invalid class: {class_name}")
                print(f"Available classes: {self.class_map.keys()}")
                raise KeyError(f"Class mapping not found: {class_name}")

            for img_path in glob.glob(os.path.join(class_path, "*.jpg")):
                self.data.append([img_path, class_name])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        img = read_image(img_path)

        # Add error handling
        try:
            class_id = self.class_map[class_name]
        except KeyError:
            print(f"Invalid class: {class_name}")
            print(f"Available classes: {self.class_map.keys()}")
            raise

        class_id = torch.tensor([class_id])

        if self.transform:
            img = self.transform(img)

        return img, class_id

In [119]:
train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([128, 128]),
    transforms.ToTensor()
    ])